In [3]:
import os
import numpy as np
import tensorflow as tf

from fxpmath import Fxp
from tensorflow.keras.optimizers import Adam

from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from qkeras import QDense, QActivation
from qkeras.quantizers import binary, ternary, quantized_relu, quantized_bits

seed = 0
width = 64
num_layers = 2
reuse_factor = 1
epochs = 5

weight_scaling_factor = 1
act_scaling_factor = 1
np.random.seed(seed)
tf.random.set_seed(seed)

os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']
saved_model_dir = f'exp_8bit_qnn'
# delete the saved_model_dir if it exists
if os.path.exists(saved_model_dir):
    import shutil
    shutil.rmtree(saved_model_dir)
# create the saved_model_dir if it doesn't exist
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)


In [4]:


# Define the fixed-point configuration
# For example, ap_fixed<16,8> corresponds to 16 total bits with 8 integer bits
total_bits = 16
integer_bits = 6
fractional_bits = total_bits - integer_bits

X_train_val = np.load('X_train_val.npy').astype(np.float32)
X_test = np.load('X_test.npy').astype(np.float32)

# convert input data to Fxp
X_train_val_fxp = Fxp(X_train_val, signed=True, n_word=total_bits, n_frac=fractional_bits).astype(np.float32)
X_test_fxp = Fxp(X_test, signed=True, n_word=total_bits, n_frac=fractional_bits).astype(np.float32)

# save the converted data
np.save('X_train_val_fxp.npy', X_train_val_fxp)
np.save('X_test_fxp.npy', X_test_fxp)

y_train_val = np.load('y_train_val.npy')
y_test = np.load('y_test.npy')
classes = np.load('classes.npy', allow_pickle=True)
print(X_train_val_fxp[:5])
print(X_test.shape)

[[-0.5107422   0.47070312  0.38476562  0.06933594 -0.04101562 -0.14160156
  -0.86816406 -0.5527344  -0.86816406 -0.7832031  -0.80859375 -0.84765625
  -0.96484375 -0.83496094  0.18652344  0.46484375]
 [ 0.7480469  -0.49804688  0.12304688  0.03613281  0.56152344  0.06640625
   0.7236328  -0.09960938  0.7236328  -0.51464844 -0.65722656 -0.5019531
  -0.19628906 -0.4345703   0.2626953  -0.8720703 ]
 [ 0.8544922  -0.90625    -1.2177734  -0.8486328  -1.1044922  -0.6230469
  -0.02929688  0.19628906 -0.02929688  0.9121094   1.1347656   1.7070312
   0.9628906   1.3779297  -1.2890625  -0.73339844]
 [ 1.2910156  -1.0869141  -1.203125   -0.84765625 -1.1044922  -0.6230469
  -0.47851562 -0.29492188 -0.47851562  0.31933594  0.90625     1.6035156
   0.42285156  0.796875   -1.2773438  -1.1484375 ]
 [-0.31933594  0.578125   -0.08984375 -0.04394531  0.71972656  0.7324219
   1.7363281   1.5029297   1.7363281  -0.703125   -1.0673828  -1.015625
  -0.38476562 -0.47558594  0.27441406 -0.27148438]]
(166000, 16)

# Construct Model
* Two layers of MLP, width = 64
* 8-bit quantization on weight and activation

In [ ]:
kernel_quantizer = quantized_bits(bits=8, integer=0)
act_quantizer = quantized_relu(bits=8, integer=0)

model = Sequential()
model.add(
   QDense(width, input_shape=(16,), name = f'fc0',
               kernel_quantizer = kernel_quantizer,
               activation = act_quantizer,
               use_bias = False)
        )
model.add(
   QDense(width, name = 'fc1',
               kernel_quantizer = kernel_quantizer,
               activation = act_quantizer,
               use_bias = False)
)
model.add(
   QDense(5, name = 'output',
               kernel_quantizer = kernel_quantizer,
               activation = act_quantizer,
               use_bias = False) 
)

model.summary()

adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss=['hinge'], metrics=['accuracy'])
callbacks = all_callbacks(
    stop_patience=1000,
    lr_factor=0.5,
    lr_patience=10,
    lr_epsilon=0.000001,
    lr_cooldown=2,
    lr_minimum=0.0000001,
    outputDir='model_3',
)
model.fit(
    X_train_val_fxp,
    y_train_val,
    batch_size=1024,
    epochs=epochs,
    validation_split=0.25,
    shuffle=True,
    callbacks=callbacks.callbacks,
)
# Save the model again but with the pruning 'stripped' to use the regular layer types
model.save(f'{saved_model_dir}/KERAS_check_best_model.h5')
